# Prepare data

## Main process
* Load raw data
* Generate X/Y, Convert to ndarray and Save
* Generate cat_cols list, Convert to int and Save
* save column names for future analysis
* Save task type: 2-class, m-class, regression

## Some illustration
* numeric feature in float64 type, categorical feature in object(str) type
* NaNs are converted to np.nan(numeric) and 'NaN'(categorical)
* split data into train/val/test in **utils.data_loader** function, because there are several possible ways of splitting

In [268]:
import pandas as pd
import os
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from io import StringIO
import re 
import pickle
import scipy.io.arff

In [269]:
# Adult, Amazon, Click prediction, KDD appetency, KDD churn, KDD upselling, HIGGS, KDD internet, Kick prediction
# San Francisco,
# Rossmann,
dataset = 'Kick prediction'  
dataset_path = "/home/v-tyan/NN_for_tabular/datasets_raw/"

dataset2type = {
'Adult': '2-class', 'Amazon': '2-class', 'Click prediction': '2-class', 'KDD appetency': '2-class', 'KDD churn': '2-class', 'KDD upselling': '2-class', 'KDD 98': '2-class', 'Kick prediction': '2-class', 'KDD internet': '2-class', 'HIGGS': '2-class',
'San Francisco': 'm-class', 
'Rossmann': 'regression', 
}

timeseries = {
'Adult': False, 'Amazon': False, 'Click prediction': False, 'KDD appetency': False, 'KDD churn': False, 'KDD upselling': False, 'KDD 98': False, 'Kick prediction': False, 'KDD internet': False, 'HIGGS': False, 
'San Francisco': True,
'Rossmann': False, 
}

## Adult

https://archive.ics.uci.edu/ml/datasets/Adult

In [270]:
if dataset == 'Adult':
    cols = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
    target_col = 'income'
    cat_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
    num_cols = list(set(cols) - {target_col} - set(cat_cols))
    train_df = pd.read_csv(os.path.join(dataset_path, 'Adult/adult.data'), sep=', ', header=None, names=cols, na_values='?', engine='python')
    test_df = pd.read_csv(os.path.join(dataset_path, 'Adult/adult.test'), sep=', ', header=None, names=cols, na_values='?', engine='python')
    test_df.replace({'<=50K.': '<=50K', '>50K.': '>50K'}, inplace=True)
    train_X, train_Y = train_df.drop(target_col, axis=1), train_df[target_col]
    test_X, test_Y = test_df.drop(target_col, axis=1), test_df[target_col]
    X, Y = train_X.append(test_X, ignore_index=True), train_Y.append(test_Y, ignore_index=True)
    columns = X.columns

## Amazon

https://www.kaggle.com/c/amazon-employee-access-challenge

In [271]:
if dataset == 'Amazon':
    df = pd.read_csv(os.path.join(dataset_path, 'Amazon/train.csv'))
    cols = list(df.columns)
    target_col = 'ACTION'
    cat_cols = list(set(set(cols) - {target_col}))
    num_cols = []  # assume all are categorial
    X, Y = df.drop(target_col, axis=1), df[target_col]
    # train_idx = pd.read_csv(os.path.join(dataset_path, "Amazon/stratified_train_idx.txt"), header=None)
    # test_idx = pd.read_csv(os.path.join(dataset_path, "Amazon/stratified_test_idx.txt"), header=None)
    # train_X, test_X, train_Y, test_Y = X.iloc[train_idx[0]], X.iloc[test_idx[0]], Y.iloc[train_idx[0]], Y.iloc[test_idx[0]]
    # X, Y = train_X.append(test_X, ignore_index=True), train_Y.append(test_Y, ignore_index=True)
    columns = X.columns

## Click prediction

https://www.kdd.org/kdd-cup/view/kdd-cup-2012-track-2

subsampling according to https://github.com/catboost/benchmarks/tree/master/quality_benchmarks

In [272]:
if dataset == 'Click prediction':
    cols = ['click', 'impression', 'url_hash', 'ad_id', 'advertiser_id', 'depth', 'position', 'query_id', 'keyword_id', 'title_id', 'description_id', 'user_id']
    target_col = 'click'
    cat_cols = ['impression', 'url_hash', 'ad_id', 'position', 'query_id', 'keyword_id', 'title_id', 'description_id']
    num_cols = list(set(cols) - {target_col} - set(cat_cols))
    with open(os.path.join(dataset_path, "Click prediction/track2/subsampling_idx.txt")) as fin:
        ids = list(map(int, fin.read().split()))
    unique_ids = set(ids)
    data_strings = {}
    with open(os.path.join(dataset_path, "Click prediction/track2/training.txt")) as fin:
        for i, string in enumerate(fin):
            if i in unique_ids:
                data_strings[i] = string
    data_rows = []
    for i in ids:
        data_rows.append(data_strings[i])
    df = pd.read_table(StringIO("".join(data_rows)), header=None, names=cols)    
    X, Y = df.drop(target_col, axis=1), df[target_col].apply(lambda x: 1 if x == 0 else -1)  # convert to 2-class
    def clean_string(s):
        return "v_" + re.sub('[^A-Za-z0-9]+', "_", str(s))
    for cat_col in cat_cols:
        X[cat_col] = X[cat_col].apply(clean_string)
    # train_idx = pd.read_csv(os.path.join(dataset_path, "Click prediction/track2/stratified_train_idx.txt"), header=None)
    # test_idx = pd.read_csv(os.path.join(dataset_path, "Click prediction/track2/stratified_test_idx.txt"), header=None)
    # train_X, test_X, train_Y, test_Y = X.iloc[train_idx[0]], X.iloc[test_idx[0]], Y.iloc[train_idx[0]], Y.iloc[test_idx[0]]
    # X, Y = train_X.append(test_X, ignore_index=True), train_Y.append(test_Y, ignore_index=True)
    columns = X.columns

## KDD appetency, churn, upselling

https://www.kdd.org/kdd-cup/view/kdd-cup-2009/Data

In [273]:
if dataset in {'KDD appetency', 'KDD churn', 'KDD upselling'}:
    df = pd.read_csv(os.path.join(dataset_path, "appetency_churn_upselling/orange_small_train.data"), sep = "\t")
    cols = list(df.columns)
    cat_cols = [cols[idx] for idx in [190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206,
                207, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228]]
    num_cols = list(set(cols) - set(cat_cols))
    dataset_ = dataset.split(' ')[-1]
    X, Y = df, -pd.read_csv(os.path.join(dataset_path, f'appetency_churn_upselling/orange_small_train_{dataset_}.labels'), header=None)[0]
    # train_idx = pd.read_csv(os.path.join(dataset_path, f'appetency_churn_upselling/{dataset_}/stratified_train_idx_{dataset_}.txt'), header=None)
    # test_idx = pd.read_csv(os.path.join(dataset_path, f'appetency_churn_upselling/{dataset_}/stratified_test_idx_{dataset_}.txt'), header=None)
    # train_X, test_X, train_Y, test_Y = X.iloc[train_idx[0]], X.iloc[test_idx[0]], Y.iloc[train_idx[0]], Y.iloc[test_idx[0]]
    columns = X.columns

## HIGGS Data Set

https://archive.ics.uci.edu/ml/datasets/HIGGS

In [274]:
if dataset == 'HIGGS':
    df = pd.read_csv(os.path.join(dataset_path, "Higgs Boson/HIGGS.csv"), header=None)
    cols = list(df.columns)
    target_col = 0
    cat_cols = []
    num_cols = list(set(cols) - {target_col} - set(cat_cols))
    X, Y = df.drop(target_col, axis=1), df[target_col]
    columns = X.columns

##  Rossmann Store Sales

https://www.kaggle.com/c/rossmann-store-sales/data

In [275]:
if dataset == 'Rossmann':
    df = pd.read_csv(os.path.join(dataset_path, "Rossmann/train.csv"))
    cols = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']
    target_col = 'Sales'
    cat_cols = ['Store', 'DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']
    num_cols = ['Customers']
    X, Y = df.drop(target_col, axis=1), df[target_col]
    columns = X.columns

## San Francisco Crime Classification

https://www.kaggle.com/c/sf-crime/data

In [276]:
if dataset == 'San Francisco':
    df = pd.read_csv(os.path.join(dataset_path, "San Francisco/train.csv"))
    cols = ['Category', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y']  # desert Dates attribute
    df = df[cols]
    df = df.reindex(list(range(len(df) - 1, -1, -1)))
    target_col = 'Category'
    cat_cols = ['DayOfWeek', 'PdDistrict', 'Address']
    num_cols = ['X', 'Y']
    X, Y = df.drop(target_col, axis=1), df[target_col]
    columns = X.columns

##  KDD internet

https://www.cs.odu.edu/~mukka/cs795sum10dm/datasets/uci-20070111/nominal/kdd_internet_usage.arff

In [277]:
if dataset == 'KDD internet':
    df = pd.read_csv(os.path.join(dataset_path, "KDD internet/kdd_internet_usage.csv"), header=None)
    cols = list(range(69))
    df = df[cols]
    target_col = 68
    cat_cols = [0, 1, 2, 11, 12, 18, 19, 20, 21, 31, 32, 33, 34, 36, 37, 38, 39, 59, 60, 61, 62]
    num_cols = list(set(cols) - {target_col} - set(cat_cols))
    Y = df[target_col].apply(lambda x: 1 if x == '0' else -1)
    X = df.drop(target_col, axis=1)
    columns = X.columns

## Kick prediction

https://www.kaggle.com/c/DontGetKicked/data

In [280]:
if dataset == 'Kick prediction':
    data = pd.read_csv(os.path.join(dataset_path, "Kick prediction/training.csv"))
    target = data["IsBadBuy"].apply(lambda x: 1.0 if x == 0 else -1.0)
    data["PurchYear"] = pd.DatetimeIndex(data['PurchDate']).year
    data["PurchMonth"] = pd.DatetimeIndex(data['PurchDate']).month
    data["PurchDay"] = pd.DatetimeIndex(data['PurchDate']).day
    data["PurchWeekday"] = pd.DatetimeIndex(data['PurchDate']).weekday
    data.drop(["RefId", "IsBadBuy", "PurchDate"], axis=1, inplace=True)
    X = data
    Y = target
    columns = X.columns
    cat_cols = [columns[i] for i in [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 23, 24, 25, 26, 27, 29, 31, 32, 33, 34]]
    num_cols = list(set(columns) - set(cat_cols))
X.shape

(72983, 35)

## Prepare numerical features

In [281]:
X[num_cols] = X[num_cols].astype(np.float64)
if dataset in {'Rossmann'}:
    Y = Y.astype(np.float64)

## Prepare category features

In [282]:
for cat_col in cat_cols:
    X[cat_col] = X[cat_col].apply(str)
if dataset in {'Adult', 'Amazon', 'Click prediction', 'KDD appetency', 'KDD churn', 'KDD internet', 'KDD upselling', 'KDD 98', 'Kick prediction'}:
    Y = Y.apply(str)

## Convert to ndarray and int

In [283]:
# if dataset in {'Adult', 'Amazon', 'Click prediction', 'KDD appetency', 'KDD churn', 'KDD upselling', 'San Francisco'}:
cat_cols_ = []
for idx, col in enumerate(X.columns):
    if col in cat_cols:
        cat_cols_.append(idx)
cat_cols = cat_cols_
X, Y = X.values, Y.values

## Save

In [284]:
data = X, Y, cat_cols, columns, dataset2type[dataset], timeseries[dataset]
save_dir = f"/home/v-tyan/NN_for_tabular/datasets/{dataset}.npy"
np.save(save_dir, data)

In [285]:
print(X.shape, Y.shape)

(72983, 35) (72983,)
